In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #dataAugmentation

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
batchsize = 8

### training dataset

#imageGeneration

In [5]:
train_datagen = ImageDataGenerator(rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,validation_split=0.2)

#working on our dataset

In [6]:
train_data = train_datagen.flow_from_directory(r'D:\Driver-Drowsiness-Detection-System\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=8, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'D:\Driver-Drowsiness-Detection-System\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=8, class_mode='categorical', subset='validation')

Found 64713 images belonging to 2 classes.
Found 16177 images belonging to 2 classes.


### testing dataset

In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_data = test_datagen.flow_from_directory(r'D:\Driver-Drowsiness-Detection-System\mrlEyes_2018_01\Prepared_Data\test', target_size=(80,80), batch_size=8, class_mode='categorical')

Found 4008 images belonging to 2 classes.


## architecture model extraction

In [8]:
#base model
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3), batch_size=8))

#head model
hmodel = bmodel.output #add the output to the base model
hmodel = Flatten()(hmodel) #flatten or compress the previous head model
hmodel = Dense(64, activation='relu')(hmodel) #densify it with only relu(important) data only
hmodel = Dropout(0.5)(hmodel) #dropout to prevent overfitting
hmodel = Dense(2, activation='softmax')(hmodel) #put those 2 neurons(classes of images) with activation of categorical data softmax

#final model - transfer learning
model = Model(inputs = bmodel.input, outputs = hmodel) #knowledge of basemodel of InceptionV3 and output of our model
for layer in bmodel.layers:
    layer.trainable = False

In [9]:
bmodel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(8, 80, 80, 3)]             0         []                            
                                                                                                  
 conv2d (Conv2D)             (8, 39, 39, 32)              864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (8, 39, 39, 32)              96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (8, 39, 39, 32)              0         ['batch_normalizati

 conv2d_11 (Conv2D)          (8, 7, 7, 32)                6144      ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (8, 7, 7, 64)                192       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (8, 7, 7, 64)                192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (8, 7, 7, 96)                288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

                                                                                                  
 activation_17 (Activation)  (8, 7, 7, 96)                0         ['batch_normalization_17[0][0]
                                                                    ']                            
                                                                                                  
 activation_18 (Activation)  (8, 7, 7, 64)                0         ['batch_normalization_18[0][0]
                                                                    ']                            
                                                                                                  
 mixed1 (Concatenate)        (8, 7, 7, 288)               0         ['activation_12[0][0]',       
                                                                     'activation_14[0][0]',       
                                                                     'activation_17[0][0]',       
          

                                                                                                  
 conv2d_28 (Conv2D)          (8, 7, 7, 96)                55296     ['activation_27[0][0]']       
                                                                                                  
 batch_normalization_28 (Ba  (8, 7, 7, 96)                288       ['conv2d_28[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_28 (Activation)  (8, 7, 7, 96)                0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_26 (Conv2D)          (8, 3, 3, 384)               995328    ['mixed2[0][0]']              
          

 conv2d_30 (Conv2D)          (8, 3, 3, 192)               147456    ['mixed3[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)          (8, 3, 3, 192)               172032    ['activation_32[0][0]']       
                                                                                                  
 conv2d_38 (Conv2D)          (8, 3, 3, 192)               172032    ['activation_37[0][0]']       
                                                                                                  
 conv2d_39 (Conv2D)          (8, 3, 3, 192)               147456    ['average_pooling2d_3[0][0]'] 
                                                                                                  
 batch_normalization_30 (Ba  (8, 3, 3, 192)               576       ['conv2d_30[0][0]']           
 tchNormalization)                                                                                
          

...

                                                                                                  
 activation_52 (Activation)  (8, 3, 3, 160)               0         ['batch_normalization_52[0][0]
                                                                    ']                            
                                                                                                  
 activation_57 (Activation)  (8, 3, 3, 160)               0         ['batch_normalization_57[0][0]
                                                                    ']                            
                                                                                                  
 average_pooling2d_5 (Avera  (8, 3, 3, 768)               0         ['mixed5[0][0]']              
 gePooling2D)                                                                                     
                                                                                                  
 conv2d_50

 batch_normalization_62 (Ba  (8, 3, 3, 192)               576       ['conv2d_62[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 batch_normalization_67 (Ba  (8, 3, 3, 192)               576       ['conv2d_67[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_62 (Activation)  (8, 3, 3, 192)               0         ['batch_normalization_62[0][0]
                                                                    ']                            
                                                                                                  
 activation_67 (Activation)  (8, 3, 3, 192)               0         ['batch_normalization_67[0][0]
          

                                                                                                  
 conv2d_71 (Conv2D)          (8, 1, 1, 320)               552960    ['activation_70[0][0]']       
                                                                                                  
 conv2d_75 (Conv2D)          (8, 1, 1, 192)               331776    ['activation_74[0][0]']       
                                                                                                  
 batch_normalization_71 (Ba  (8, 1, 1, 320)               960       ['conv2d_71[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 batch_normalization_75 (Ba  (8, 1, 1, 192)               576       ['conv2d_75[0][0]']           
 tchNormalization)                                                                                
          

                                                                                                  
 activation_82 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_82[0][0]
                                                                    ']                            
                                                                                                  
 activation_83 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_83[0][0]
                                                                    ']                            
                                                                                                  
 batch_normalization_84 (Ba  (8, 1, 1, 192)               576       ['conv2d_84[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activatio

                                                                                                  
 activation_88 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_88[0][0]
                                                                    ']                            
                                                                                                  
 activation_91 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_91[0][0]
                                                                    ']                            
                                                                                                  
 activation_92 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_92[0][0]
                                                                    ']                            
                                                                                                  
 batch_nor

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(8, 80, 80, 3)]             0         []                            
                                                                                                  
 conv2d (Conv2D)             (8, 39, 39, 32)              864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (8, 39, 39, 32)              96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (8, 39, 39, 32)              0         ['batch_normalization[0][0

 conv2d_11 (Conv2D)          (8, 7, 7, 32)                6144      ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (8, 7, 7, 64)                192       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (8, 7, 7, 64)                192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (8, 7, 7, 96)                288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

                                                                                                  
 activation_17 (Activation)  (8, 7, 7, 96)                0         ['batch_normalization_17[0][0]
                                                                    ']                            
                                                                                                  
 activation_18 (Activation)  (8, 7, 7, 64)                0         ['batch_normalization_18[0][0]
                                                                    ']                            
                                                                                                  
 mixed1 (Concatenate)        (8, 7, 7, 288)               0         ['activation_12[0][0]',       
                                                                     'activation_14[0][0]',       
                                                                     'activation_17[0][0]',       
          

                                                                                                  
 conv2d_28 (Conv2D)          (8, 7, 7, 96)                55296     ['activation_27[0][0]']       
                                                                                                  
 batch_normalization_28 (Ba  (8, 7, 7, 96)                288       ['conv2d_28[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_28 (Activation)  (8, 7, 7, 96)                0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_26 (Conv2D)          (8, 3, 3, 384)               995328    ['mixed2[0][0]']              
          

 conv2d_30 (Conv2D)          (8, 3, 3, 192)               147456    ['mixed3[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)          (8, 3, 3, 192)               172032    ['activation_32[0][0]']       
                                                                                                  
 conv2d_38 (Conv2D)          (8, 3, 3, 192)               172032    ['activation_37[0][0]']       
                                                                                                  
 conv2d_39 (Conv2D)          (8, 3, 3, 192)               147456    ['average_pooling2d_3[0][0]'] 
                                                                                                  
 batch_normalization_30 (Ba  (8, 3, 3, 192)               576       ['conv2d_30[0][0]']           
 tchNormalization)                                                                                
          

...

                                                                                                  
 activation_52 (Activation)  (8, 3, 3, 160)               0         ['batch_normalization_52[0][0]
                                                                    ']                            
                                                                                                  
 activation_57 (Activation)  (8, 3, 3, 160)               0         ['batch_normalization_57[0][0]
                                                                    ']                            
                                                                                                  
 average_pooling2d_5 (Avera  (8, 3, 3, 768)               0         ['mixed5[0][0]']              
 gePooling2D)                                                                                     
                                                                                                  
 conv2d_50

 batch_normalization_62 (Ba  (8, 3, 3, 192)               576       ['conv2d_62[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 batch_normalization_67 (Ba  (8, 3, 3, 192)               576       ['conv2d_67[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_62 (Activation)  (8, 3, 3, 192)               0         ['batch_normalization_62[0][0]
                                                                    ']                            
                                                                                                  
 activation_67 (Activation)  (8, 3, 3, 192)               0         ['batch_normalization_67[0][0]
          

                                                                                                  
 conv2d_71 (Conv2D)          (8, 1, 1, 320)               552960    ['activation_70[0][0]']       
                                                                                                  
 conv2d_75 (Conv2D)          (8, 1, 1, 192)               331776    ['activation_74[0][0]']       
                                                                                                  
 batch_normalization_71 (Ba  (8, 1, 1, 320)               960       ['conv2d_71[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 batch_normalization_75 (Ba  (8, 1, 1, 192)               576       ['conv2d_75[0][0]']           
 tchNormalization)                                                                                
          

                                                                                                  
 activation_82 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_82[0][0]
                                                                    ']                            
                                                                                                  
 activation_83 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_83[0][0]
                                                                    ']                            
                                                                                                  
 batch_normalization_84 (Ba  (8, 1, 1, 192)               576       ['conv2d_84[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activatio

                                                                                                  
 activation_88 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_88[0][0]
                                                                    ']                            
                                                                                                  
 activation_91 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_91[0][0]
                                                                    ']                            
                                                                                                  
 activation_92 (Activation)  (8, 1, 1, 384)               0         ['batch_normalization_92[0][0]
                                                                    ']                            
                                                                                                  
 batch_nor

### save the best model

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [12]:
#where you want to save your model
checkpoint = ModelCheckpoint(r'D:\Driver-Drowsiness-Detection-System\models\model.h5', monitor='val_loss', save_best_only=True, verbose=3) 

#when you are training for a long time, accuracy not inc, val_loss not dec then stop the training
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

#val_loss is not dec after 2-3 epochs(iterations) reduce the learning rate
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

#callbacks
callbacks=[checkpoint, earlystop, learning_rate]

### compile the model

In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

### training of model

In [14]:
model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize, 
                    validation_data=validation_data, 
                    validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks, epochs=5)

C:\Users\sonip\AppData\Local\Temp\ipykernel_5156\3167212791.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
8089/8089 [==============================] - ETA: 0s - loss: 0.2040 - accuracy: 0.9212
Epoch 1: val_loss improved from inf to 0.18683, saving model to D:\Driver-Drowsiness-Detection-System\models\model.h5


D:\anaconda\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8089/8089 [==============================] - 594s 73ms/step - loss: 0.2040 - accuracy: 0.9212 - val_loss: 0.1868 - val_accuracy: 0.9201 - lr: 0.0010
Epoch 2/5
8089/8089 [==============================] - ETA: 0s - loss: 0.1742 - accuracy: 0.9315
Epoch 2: val_loss improved from 0.18683 to 0.17059, saving model to D:\Driver-Drowsiness-Detection-System\models\model.h5
8089/8089 [==============================] - 618s 76ms/step - loss: 0.1742 - accuracy: 0.9315 - val_loss: 0.1706 - val_accuracy: 0.9284 - lr: 0.0010
Epoch 3/5
8089/8089 [==============================] - ETA: 0s - loss: 0.1698 - accuracy: 0.9340
Epoch 3: val_loss did not improve from 0.17059
8089/8089 [==============================] - 591s 73ms/step - loss: 0.1698 - accuracy: 0.9340 - val_loss: 0.1837 - val_accuracy: 0.9240 - lr: 0.0010
Epoch 4/5
8089/8089 [==============================] - ETA: 0s - loss: 0.1623 - accuracy: 0.9363
Epoch 4: val_loss did not improve from 0.17059
8089/8089 [==============================] - 8

### model evaluation

In [15]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\sonip\AppData\Local\Temp\ipykernel_5156\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.1385682374238968
0.9470739960670471


In [16]:
acc_vd, loss_vd = model.evaluate_generator(validation_data)
print(acc_vd)
print(loss_vd)

C:\Users\sonip\AppData\Local\Temp\ipykernel_5156\138623123.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vd, loss_vd = model.evaluate_generator(validation_data)


0.18649640679359436
0.9222352504730225


In [17]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

C:\Users\sonip\AppData\Local\Temp\ipykernel_5156\3515202625.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.5610940456390381
0.8148702383041382
